In [ ]:
#Select GPU in Colab Configurations
!nvidia-smi

In [ ]:
!pip install plotly q

In [ ]:
!git clone https://github.com/openai/shap-e.git

In [ ]:
!pip install -e ./shap-e/

In [ ]:
%cd shap-e/
!pwd

In [ ]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

Add a text description in 'prompt' variable

In [ ]:
batch_size = 4
guidance_scale = 15.0
prompt = "a shark"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

In [ ]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAADMAAAAzAAAAMzMAMzMzMzMzZmYzZjNmZmZmZpmZZjMzmW…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAADMAAAAzAAAAMwAzMzMzMzMzZmYzZgBmZjNmZmZmZpmZZj…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAADMAAAAzAAAAMwAzMzMzMwAzZjMzZjNmZmZmZjMzmTNmmW…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAAAAzAAAAMwAzMzMzMwAzZjMzZgBmZjNmZjNmmWZmmWaZmZ…

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)

# The Below Code is for Visualizing 3D model inside Google Colab

In [ ]:
!pip install open3d

In [ ]:
import open3d as o3d
import numpy as np
import plotly.graph_objects as go

mesh = o3d.io.read_triangle_mesh("example_mesh_0.obj")
if mesh.is_empty(): exit()

In [ ]:
if not mesh.has_vertex_normals(): mesh.compute_vertex_normals()
if not mesh.has_triangle_normals(): mesh.compute_triangle_normals()

In [ ]:
triangles = np.asarray(mesh.triangles)
vertices = np.asarray(mesh.vertices)
colors = None
if mesh.has_triangle_normals():
    colors = (0.5, 0.5, 0.5) + np.asarray(mesh.triangle_normals) * 0.5
    colors = tuple(map(tuple, colors))
else:
    colors = (1.0, 0.0, 0.0)

In [ ]:
fig = go.Figure(
    data=[
        go.Mesh3d(
            x=vertices[:,0],
            y=vertices[:,1],
            z=vertices[:,2],
            i=triangles[:,0],
            j=triangles[:,1],
            k=triangles[:,2],
            facecolor=colors,
            opacity=0.50)
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)


In [ ]:
fig.show(renderer="colab")